# Assignment Part 1

In [ ]:
import pandas as pd # Import Pandas

In [3]:
pip install lxml html5lib beautifulsoup4 # Install Beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") # Scrape data from Wikipedia

In [12]:
PC_df = df[0]
PC_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [132]:
# Remove "Not assigned"
PC_df.drop(PC_df[PC_df["Borough"] == "Not assigned"].index, inplace=True)
PC_df.reset_index(drop=True, inplace=True)
PC_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [150]:
# Group by Postal code, join the neighbourhoods
PC_df_2 = pd.DataFrame(PC_df.groupby(["Postal Code","Borough"])["Neighbourhood"].apply(lambda x:",".join(x)))
PC_df_2.reset_index(inplace=True)
PC_df_2.head(20)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [140]:
# Check if any neighborhoods are not assigned
PC_df_2[PC_df_2["Neighbourhood"]=="Not assigned"].index

Int64Index([], dtype='int64')

In [88]:
print("No unassigned Neighbourhoods")

No unassigned Neighbourhoods


In [122]:
PC_df_2.shape

(103, 1)

# Assignment Part 2

In [183]:
# The code was removed by Watson Studio for sharing.

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [185]:
column_names = ["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"] 

neighbourhoods = pd.DataFrame(columns = column_names)
for i in PC_df_2.index:
    neighbourhoods = neighbourhoods.append({"Postal Code":PC_df_2.loc[i]["Postal Code"],
                                           "Borough": PC_df_2.loc[i]["Borough"],
                                           "Neighbourhood":PC_df_2.loc[i]["Neighbourhood"],
                                           "Latitude":df_data_1.loc[PC_df_2.loc[i]["Postal Code"]]["Latitude"],
                                           "Longitude":df_data_1.loc[PC_df_2.loc[i]["Postal Code"]]["Longitude"], }, ignore_index = True)
neighbourhoods.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Assignment Part 3

The cell below contains the definition of the access tokens for the location service

In [326]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: WVDLAWQSPRWK1OE3NFXKNGWFOVMNXK51RIIVHOIMAVC5RPSA
CLIENT_SECRET:JHWV2B2EJD0VMS3JK4GZNEL0RUUQ10WXMFTBS2AIUUF0G1PZ


In [312]:
!pip install folium # Install folium
import folium
import requests
import json
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

##### Get Neighbourhood Info from foursquare with function defined in lab

In [204]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [229]:
# Get all neighbourhoods which have "Toronto" in them

torontohoods = neighbourhoods[neighbourhoods["Borough"].str.contains("Toronto")]
torontohoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [230]:
toronto_venues = getNearbyVenues(names=torontohoods["Neighbourhood"], 
                                 latitudes = torontohoods["Latitude"],
                                 longitudes = torontohoods["Longitude"]
                                )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [231]:
toronto_venues.head() #check if output is plausible

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [232]:
toronto_venues.groupby("Neighbourhood").count().head() # show the number of venues in each neighbourhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30


### Generate Input for clustering (based on venue category)
Weighted venue catogories in each neighbourhood as an input for the clustering algorithm

In [233]:
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")

toronto_onehot["Neighbourhood"] = toronto_venues["Neighbourhood"]
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [302]:
toronto_grouped = toronto_onehot.groupby("Neighbourhood").mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.033333


### Clustering of the Neighbourhoods based on the venues

In [305]:
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop("Neighbourhood", 1)

kmeans = KMeans(n_clusters=kclusters, random_state = 1).fit(toronto_grouped_clustering)
toronto_grouped["Cluster"]=kmeans.labels_
toronto_grouped.set_index("Neighbourhood", inplace=True)
toronto_grouped.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Cluster
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,2
"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,3
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.058824,2
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0
Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.033333,0


In [318]:
# Add the cluster labels from the KMeans clustering to the torontohoods, so they can be visualized in folium
colum_titles = [ "Neighbourhood", "Cluster","Latitude", "Longitude"]
merged_torontohoods = pd.DataFrame(columns = colum_titles)

for i in torontohoods.index:
    merged_torontohoods = merged_torontohoods.append({"Neighbourhood":torontohoods.loc[i]["Neighbourhood"],
                                                      "Latitude":torontohoods.loc[i]["Latitude"],
                                                      "Longitude":torontohoods.loc[i]["Longitude"],
                                                      "Cluster":int(toronto_grouped.loc[torontohoods.loc[i]["Neighbourhood"]]["Cluster"])
 }, ignore_index = True)
merged_torontohoods.head(5)


,Neighbourhood,Cluster,Latitude,Longitude
0,The Beaches,7,43.676357,-79.293031
1,"The Danforth West, Riverdale",3,43.679557,-79.352188
2,"India Bazaar, The Beaches West",3,43.668999,-79.315572
3,Studio District,0,43.659526,-79.340923
4,Lawrence Park,4,43.728020,-79.388790


In [325]:
# Visualize in Folien 
# create map
latitude = 43.651070 #Location of Toronto
longitude = -79.347015 # Location of Toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_torontohoods['Latitude'], merged_torontohoods['Longitude'], merged_torontohoods['Neighbourhood'], merged_torontohoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters